In [1]:
from yfapi import YahooFinanceAPI, Interval
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine
from decouple import config
import pandas as pd
import logging
import boto3
from botocore.exceptions import ClientError
import os

In [2]:
# instantiate the API and set the interval to weekly data
dh = YahooFinanceAPI(Interval.DAILY)

# get datime of yesterday at 00:00:00
yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
now = datetime.strptime(yesterday, "%Y, %m, %d")
then = datetime(2022, 3, 12)

In [3]:
#dbuname = 'Christoph'
#dbpwd = 'dbyahoo1'
#host = 'dbyf1.c8vnebtzzltk.us-east-1.rds.amazonaws.com'
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = '5432'
dbname = 'dbyf1'
print(dbuname)
print(dbpwd)
print(host)

Christoph
dbyahoo1
dbyf1.c8vnebtzzltk.us-east-1.rds.amazonaws.com


In [4]:
s3_client =boto3.client('s3')
s3_bucket_name='s3dwlcs'
s3 = boto3.resource('s3', aws_access_key_id=config('AWSAKID'),
                    aws_secret_access_key=config('AWSSAK'),
                    aws_session_token=config('AWSST'))

In [10]:
my_bucket=s3.Bucket(s3_bucket_name)

for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='s3dwlcs', key='yf_euro_stoxx.csv')
s3.ObjectSummary(bucket_name='s3dwlcs', key='yf_gold.csv')
s3.ObjectSummary(bucket_name='s3dwlcs', key='yf_msci_world.csv')
s3.ObjectSummary(bucket_name='s3dwlcs', key='yf_msci_world_test.csv')
s3.ObjectSummary(bucket_name='s3dwlcs', key='yf_silver.csv')


In [6]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        #response = s3_client.upload_file(file_name, bucket, object_name)
        s3.Bucket(bucket).upload_file(file_name, file_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [8]:
labels={'GC=F':'gold','SI=F':'silver','URTH':'msci_world', 'EXW1.DE':'euro_stoxx', 'CSSMI.SW':'smi', 'EXXT.DE':'nasdaq' }

In [7]:
def fetch_load_data(instrument, start, end):
    
    while True:    
        #fetch data
        try:
            df = dh.get_ticker_data(instrument, start, end)
            file_name = 'yf_'+labels[instrument]+'.csv'
            print(file_name)
            df['Ins'] = instrument
            df['Ins_label'] = labels[instrument]
            df.to_csv(file_name)
        except:
            print("Investment Instrument not available: " + instrument)
            break
                            
        #load data
        try:
            bucket_name = 's3dwlcs'
            upload_file(file_name, bucket_name)
        except:
            print("Data load failed: " + file_name)   
            break
        break             
            

In [9]:
#inv_instruments = ['URTH', 'EXW1.DE', 'CSSMI.SW', 'EXXT.DE', 'GC=F', 'SI=F']
inv_instruments = ['EXW1.DE']

for ins in inv_instruments:
    fetch_load_data(ins, then, now)


yf_euro_stoxx.csv
